In [1]:
import torch
t = torch.randint(10, (5,))
t

tensor([2, 1, 9, 1, 2])

In [26]:
N = 1
L = 2400
C = 600
D = 256
source = torch.randperm(N*L*D).view(N, L, D).float().cuda()
t = torch.linspace(
    start=0, end=(N-1)*L, 
    steps=N, dtype=torch.int32
)  
t = t[:, None, None].repeat(1, L, C)
t1 = torch.randint(
    low=0, 
    high=L, 
    size=(N, L, C)
) 
ind_global = (t1 + t).flatten(0, -1)
output = source.flatten(0, 1)[ind_global, :].view(N, L, -1, D)

In [46]:
import torch

# 假设我们有以下变量
batch_size = 4
num_points = 3
max_correspondences = 5  # 这是经过检查我们知道的最大对应点数量

# 假设的每个点的对应点数量
num_correspondences_per_point = torch.tensor([2, 3, 1])

# 创建一个批次中所有点对应的数量的扩展张量
expanded_correspondences = num_correspondences_per_point[None, :, None].expand(batch_size, -1, max_correspondences)

# 创建一个范围张量，范围是0到最大对应点数
range_tensor = torch.arange(max_correspondences)[None, None, :].expand_as(expanded_correspondences)

# 使用比较操作来创建掩码
mask = range_tensor < expanded_correspondences

# 使用掩码来填充索引，我们使用masked_fill来放置-1
padded_indices = torch.full(expanded_correspondences.shape, -1)  # 填充-1
padded_indices[mask] = range_tensor[mask]

print("Padded indices:\n", padded_indices)
print("Mask:\n", mask)


Padded indices:
 tensor([[[ 0,  1, -1, -1, -1],
         [ 0,  1,  2, -1, -1],
         [ 0, -1, -1, -1, -1]],

        [[ 0,  1, -1, -1, -1],
         [ 0,  1,  2, -1, -1],
         [ 0, -1, -1, -1, -1]],

        [[ 0,  1, -1, -1, -1],
         [ 0,  1,  2, -1, -1],
         [ 0, -1, -1, -1, -1]],

        [[ 0,  1, -1, -1, -1],
         [ 0,  1,  2, -1, -1],
         [ 0, -1, -1, -1, -1]]])
Mask:
 tensor([[[ True,  True, False, False, False],
         [ True,  True,  True, False, False],
         [ True, False, False, False, False]],

        [[ True,  True, False, False, False],
         [ True,  True,  True, False, False],
         [ True, False, False, False, False]],

        [[ True,  True, False, False, False],
         [ True,  True,  True, False, False],
         [ True, False, False, False, False]],

        [[ True,  True, False, False, False],
         [ True,  True,  True, False, False],
         [ True, False, False, False, False]]])


In [30]:
import torch
from variable_gather import gather_index

if torch.cuda.is_available():
    # 创建两个CUDA事件
    start_event = torch.cuda.Event(enable_timing=True)
    end_event = torch.cuda.Event(enable_timing=True)

    # 初始化要在GPU上执行的张量
    x = torch.randn((4, 4800, 4800), device="cuda") > 0.5

    # 记录开始时间
    start_event.record()

    # 执行某个CUDA操作
    for i in range(100):
        output = gather_index(x, 880)
    
    # 记录结束时间
    end_event.record()

    # 等待事件完成
    torch.cuda.synchronize()  # 等待所有CUDA核心完成当前所有已提交的命令

    # 计算执行时间
    elapsed_time_ms = start_event.elapsed_time(end_event) / 100

    print(f"Execution time: {elapsed_time_ms} ms")
else:
    print("CUDA is not available.")


Execution time: 11.329454345703125 ms


In [30]:
x = torch.randn((2, 1200, 1200), device="cuda").flatten(0, 1) > 0.5
output = torch.zeros((2, 1200, 440), device=x.device).flatten(0, 1)
for i in range(2 * 1200):
    count = 0;
    for j in range(1200):
        if (x[i][j] == True):
            output[i, count] = j; 
            count += 1;
        if count >= 440:
            break


In [28]:
import torch

def test():
    # 假定输入x
    x = torch.randn((4, 4800, 4800), device="cuda").flatten(0, 1) > 0.5
    
    # 创建一个索引张量，大小与x相同
    indices = torch.arange(4800, device=x.device).expand_as(x)

    # 使用x作为掩码，选取True值的索引
    true_indices = torch.where(x, indices, torch.tensor(4800, device=x.device))

    # 对true_indices进行排序，因为torch.where不能保证顺序，并且需要去除填充的1200值
    sorted_indices, _ = torch.sort(true_indices, dim=1)

    # 确保每一行最多只有440个有效索引
    # 注意：由于先前步骤中填充的是1200，排序后有效索引将排在前面
    output = sorted_indices[:, :880]

    # 恢复output的形状以匹配原始需求
    output = output.reshape(4, 4800, 880)
    return output

output = test()


In [29]:
import torch
from variable_gather import gather_index

if torch.cuda.is_available():
    # 创建两个CUDA事件
    start_event = torch.cuda.Event(enable_timing=True)
    end_event = torch.cuda.Event(enable_timing=True)


    # 记录开始时间
    start_event.record()

    # 执行某个CUDA操作
    for i in range(100):
        output = test()

    # 记录结束时间
    end_event.record()

    # 等待事件完成
    torch.cuda.synchronize()  # 等待所有CUDA核心完成当前所有已提交的命令

    # 计算执行时间
    elapsed_time_ms = start_event.elapsed_time(end_event) / 100

    print(f"Execution time: {elapsed_time_ms} ms")
else:
    print("CUDA is not available.")

Execution time: 117.86421875 ms


In [1]:
import torch

In [7]:
mask = torch.rand((2, 10, 10), device='cuda:0') > 0.5

In [11]:
b_ids, i_ids, j_ids = torch.where(mask)
num_gt_b = torch.sum(mask, dim=(1, 2))

In [12]:
b_ids, num_gt_b

(tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0'),
 tensor([50, 40], device='cuda:0'))

In [22]:
# idx_cumsum = torch.cumsum(num_gt_b, 0)
idx_cumsum = num_gt_b.cumsum(dim=0)

In [27]:
bs = idx_cumsum.shape[0]
for idx in range(bs):
    low = 0 if idx == 0 else idx_cumsum[idx-1]
    high = idx_cumsum[idx]
    

0-50

50-90



In [28]:
from kornia.geometry.epipolar import find_essential, decompose_essential_matrix

# padded gt matches [N, PAD_NUM_MIN, 2]
index0 = torch.randint(0, 50, (32, ))
index1 = torch.zeros((32, ))
index = torch.stack([index0, index1], dim=0)


In [29]:
index.shape

torch.Size([2, 32])

In [2]:
import torch
N = 4
C = 32
spv_b_ids = torch.randperm(100, dtype=torch.int64)
pad_index = torch.zeros((4, 32))
drop = torch.tensor([True, False, True, True])
b_ids = torch.zeros((N, C), device='cuda:0')
b_ids[~drop, :8] = spv_b_ids[pad_index[~drop, :8].flatten(0, 1)].view(-1, 8)  # [3, 8] 
i_ids[~drop, :8] = spv_i_ids[pad_index[~drop, :8].flatten(0, 1)].view(-1, 8)  # [3, 8] 
l_ids[~drop, :8] = spv_j_ids[pad_index[~drop, :8].flatten(0, 1)].view(-1, 8)  # [3, 8] 

SyntaxError: invalid syntax (1964273117.py, line 8)

In [12]:
a = torch.linspace(0, 3, 4).unsqueeze(dim=-1).repeat(1, 32)
a.shape

torch.Size([4, 32])

In [4]:
import torch

# 示例维度
N, L = 2, 5  # N: 批次大小, L: 每个批次的点数
D = 64
feat = torch.randn((4, 1200, 512))
index = torch.randint(0, 1200, (4, 1200, 200))
print(points)
indices = torch.randint(L, (N, 10))  # 随机生成的需要提取的点的索引
print(indices)
# 使用高级索引提取点
selected_points = points[torch.arange(N).unsqueeze(1), indices]
selected_points

tensor([[5, 6, 9, 3, 4],
        [1, 7, 8, 0, 2]])
tensor([[2, 4, 0, 1, 4, 4, 0, 4, 4, 2],
        [0, 3, 3, 3, 3, 4, 4, 4, 2, 0]])


tensor([[9, 4, 5, 6, 4, 4, 5, 4, 4, 9],
        [1, 0, 0, 0, 0, 2, 2, 2, 8, 1]])

In [20]:
import torch

feat_all = torch.ones((4, 5), requires_grad=True)
feat_all_out = feat_all * 2
skip_sample = torch.tensor([False, True, False, False])
feat = feat_all_out[~skip_sample]
feat_out = 2 * feat
feat_restore = torch.zeros_like(feat_all)
feat_restore[~skip_sample] = feat_out
feat_restore[skip_sample] = feat_all_out[skip_sample]
feat_restore.backward(torch.ones_like(feat_restore))
feat_all_out.grad

/tmp/ipykernel_9495/3982700266.py:12: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400400184/work/build/aten/src/ATen/core/TensorBody.h:489.)
  feat_all_out.grad


In [1]:
from kornia.utils import create_meshgrid

kkgrid = create_meshgrid(60, 80, False, 'cuda', torch.int64)

NameError: name 'torch' is not defined